In [16]:
%pip install geopy
%pip install tenacity

from geopy.geocoders import Nominatim
import pandas as pd
from geopy.exc import GeocoderTimedOut
from tenacity import retry, stop_after_attempt, wait_exponential

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
# Function to get latitude and longitude from address with retries
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def get_lat_long(address):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(address, timeout=10)  # Increase timeout to 10 seconds
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Function to update latitude and longitude if either is 0
def update_lat_long(row):
    if row['Latitude'] == 0 or row['Longitude'] == 0:
        # Construct the address from address components
        address = (
            f"{row['Street Number']} {row['Street Name']}, "
            f"{row['City']}, {row['State']} {row['Zip Code']}"
        )
        try:
            # Get new latitude and longitude with retry mechanism
            new_lat, new_long = get_lat_long(address)
            if new_lat is not None and new_long is not None:
                # Update the latitude and longitude values
                row['Latitude'] = new_lat
                row['Longitude'] = new_long
        except Exception as e:
            print(f"Error geocoding address: {address} - {e}")
    return row

# Path to the cleaned CSV file with addresses
cleaned_csv_path = "raw_cleaned.csv"

# Load the cleaned CSV file into a DataFrame
df_cleaned = pd.read_csv(cleaned_csv_path)

# Apply the update_lat_long function to the DataFrame
df_updated = df_cleaned.apply(update_lat_long, axis=1)

# Save the updated DataFrame back to a CSV file
updated_df_path = "updated_dataframe.csv"
df_updated.to_csv(updated_df_path, index=False)

print("Latitude and Longitude values updated where necessary.")

In [ ]:
# # Function to get latitude and longitude from address
# def get_lat_long(address):
#     geolocator = Nominatim(user_agent="my_geocoder")
#     location = geolocator.geocode(address, timeout=10)  # Increase timeout (e.g., 10 seconds)
#     if location:
#         return location.latitude, location.longitude
#     else:
#         return None, None


# # Path to the cleaned CSV file with addresses
# cleaned_csv_path = "raw_cleaned.csv"

# # Load the cleaned CSV file into a DataFrame
# df_cleaned = pd.read_csv(cleaned_csv_path)

# # Create a new DataFrame to store latitude and longitude
# df_with_coordinates = df_cleaned.copy()

# # Construct the address string from address components
# df_with_coordinates['Address'] = df_with_coordinates['Street Number'].astype(str) + ' ' + \
#                                  df_with_coordinates['Street Name'] + ', ' + \
#                                  df_with_coordinates['City'] + ', ' + \
#                                  df_with_coordinates['State'] + ' ' + \
#                                  df_with_coordinates['Zip Code'].astype(str)

# # Apply the get_lat_long function to obtain latitude and longitude
# df_with_coordinates['get_Latitude'], df_with_coordinates['get_Longitude'] = zip(*df_with_coordinates['Address'].apply(get_lat_long))

# # Save the DataFrame with latitude and longitude to a new CSV file
# coordinates_csv_path = "addresses_with_coordinates.csv"
# df_with_coordinates.to_csv(coordinates_csv_path, index=False, mode='w')

# print('Latitude and longitude coordinates obtained and saved.')
# print('***********************************************************')
# print(df_with_coordinates.head())

KeyboardInterrupt: 